In [1]:
# we use bokeh for plotting
from bokeh.io import output_notebook, show
from bokeh.layouts import column
from decimal import Decimal

output_notebook()

Loading BokehJS ...

How Uniswap works
---

![uniswap](https://docs.uniswap.org/assets/images/anatomy-82d82239e5417e36ca9da17d14961434.jpg)

Uniswap is an automated liquidity protocol powered by a constant product formula and implemented in a system of non-upgradeable smart contracts on the Ethereum blockchain. It obviates the need for trusted intermediaries, prioritizing decentralization, censorship resistance, and security. Uniswap is open-source software licensed under the GPL.

Each Uniswap smart contract, or pair, manages a liquidity pool made up of reserves of two ERC-20 tokens.

Anyone can become a liquidity provider (LP) for a pool by depositing an equivalent value of each underlying token in return for pool tokens. These tokens track pro-rata LP shares of the total reserves, and can be redeemed for the underlying assets at any time.

In [2]:
from cpy_amm.swap import Pool, constant_product_swap
from cpy_amm.plotting import new_pool_figure

# liquidity pool made up of reserves of Token A 
pool_token_A = Pool("A", 100)
# liquidity pool made up of reserves of Token B 
pool_token_B = Pool("B", 100)
# the constant product formula
k = pool_token_A.balance*pool_token_B.balance
# amount of tokens A to swap in
swap_input = 10
# swap 10 tokens A for tokens B
constant_product_swap(swap_input, pool_token_A, pool_token_B, k) 
# plotting the reserves before and after swap 
p = new_pool_figure(pool_token_A, pool_token_B, steps=["Before Swap", "After Swap"])
# display plot
show(column(p, sizing_mode="stretch_both"))

![constant product](https://docs.uniswap.org/assets/images/trade-2027cdc01fe7c448f60a5e7da34af9b9.jpg)

Pairs act as automated market makers, standing ready to accept one token for the other as long as the "constant product" formula is preserved. This formula, most simply expressed as `x * y = k`, states that trades must not change the product (`k`) of a pair’s reserve balances (`x` and `y`). Because `k` remains unchanged from the reference frame of a trade, it is often referred to as the invariant. This formula has the desirable property that larger trades (relative to reserves) execute at exponentially worse rates than smaller ones.

Because the relative price of the two pair assets can only be changed through trading, divergences between the Uniswap price and external prices create arbitrage opportunities. This mechanism ensures that Uniswap prices always trend toward the market-clearing price.

In [3]:
from cpy_amm.plotting import new_price_impact_figure

# liquidity pool made up of reserves of Token A 
pool_token_A = Pool("A", 1200)
# liquidity pool made up of reserves of Token B 
pool_token_B = Pool("B", 400)
# the constant product formula
k = pool_token_A.balance*pool_token_B.balance
# new plot with price impact of swapping 3 tokens A
p = new_price_impact_figure(pool_token_A, pool_token_B, dx=3)
# display plot
show(column(p, sizing_mode="stretch_both"))

First liquidity provider
---

Each Uniswap liquidity pool is a trading venue for a pair of ERC20 tokens. When a pool contract is created, its balances of each token are 0; in order for the pool to begin facilitating trades, someone must seed it with an initial deposit of each token.

The first liquidity provider is the one who sets the initial price of the pool. They are incentivized to deposit an equal value of both tokens into the pool. To see why, consider the case where the first liquidity provider deposits tokens at a ratio different from the current market rate. This immediately creates a profitable arbitrage opportunity, which is likely to be taken by an external party.  

In [4]:
from cpy_amm.swap import init_liquidity, MarketQuote, constant_product_swap
from cpy_amm.bokeh_swap import new_constant_product_figure, new_pool_figure

# initial liquidity in USD
deposit_usd = 10000
# USDC/USD market price
usdt_usd = MarketQuote("USDT/USD", 1)
# ETH/USD market price
uni_usd = MarketQuote("UNI/USD", 6.32)
# initialize 2 pools given initial deposit and market quotes, all in USD
usdt_pool, uni_pool, k = init_liquidity(deposit_usd, usdt_usd, uni_usd)
# new plot with constant product curve
cp_plot = new_constant_product_figure(usdt_pool, uni_pool, x_min=500, x_max=15000)
# plotting the reserves before and after swap 
pool_at_t0 = new_pool_figure(usdt_pool, uni_pool, steps=["Initial deposit"])
# swap 3000 USDT for UNI
constant_product_swap(3000, usdt_pool, uni_pool, k) 
# plotting the reserves before and after the swap 
pool_at_t1 = new_pool_figure(usdt_pool, uni_pool, steps=["Before Swap", "After Swap"])
# display plots
show(column(cp_plot, pool_at_t0, pool_at_t1, sizing_mode="stretch_both"))

#### Using the same pools for the pair UNI/USDT

In [5]:
from cpy_amm.swap import reset_pools
from cpy_amm.plotting import new_price_impact_figure

# reset pools to the initial deposit 
reset_pools(usdt_pool, uni_pool)
# new plot with constant product curve for UNI/USDT
cp_plot = new_constant_product_figure(uni_pool, usdt_pool, x_min=100, x_max=3000)
# order size for UNI eg. we're seller UNI buyer USDT
# adding price impact to the swap for a sell order of 500 UNI tokens
cp_plot = new_price_impact_figure(uni_pool, usdt_pool, dx=500)
# swap 500 UNI for USDT
constant_product_swap(500, uni_pool, usdt_pool, k) 
# plotting the reserves before and after swap 
pool_plot = new_pool_figure(usdt_pool, uni_pool, steps=["Before Swap", "After Swap"])
# display plots
show(column(cp_plot, pool_plot, sizing_mode="stretch_both"))

Providing Liquidity
---

When providing liquidity from a smart contract, the most important thing to keep in mind is that tokens deposited into a pool at any rate other than the current reserve ratio are vulnerable to being arbitraged. As an example, if the ratio of x:y in a pair is 10:2 (i.e. the price is 5), and someone naively adds liquidity at 5:2 (a price of 2.5), the contract will simply accept all tokens (changing the price to 3.75 and opening up the market to arbitrage), but only issue pool tokens entitling the sender to the amount of assets sent at the proper ratio, in this case 5:1. To avoid donating to arbitrageurs, it is imperative to add liquidity at the current price. Luckily, it's easy to ensure that this condition is met!



In [6]:
from cpy_amm.swap import init_liquidity, add_liquidity
from cpy_amm.plotting import new_constant_product_figure, new_price_impact_figure

# USDC/USD market price
usdt_usd = MarketQuote("USDT/USD", 1)
# ETH/USD market price
uni_usd = MarketQuote("UNI/USD", 6.32)
# initialize 2 pools given initial deposit of 10000$ and market quotes, all in USD
usdt_pool, uni_pool, k = init_liquidity(10000, usdt_usd, uni_usd)
# new plot with constant product curve
cp_plot = new_constant_product_figure(usdt_pool, uni_pool)
# deposit 5000$ 3 times, mid price must not change 
for _ in range(0,3):
    # add 5000$ liquidity using USD market quote for each token 
    add_liquidity(usdt_pool, uni_pool, 5000, usdt_usd, uni_usd)
    # new plot with constant product curve
    cp_plot = new_constant_product_figure(usdt_pool, uni_pool, bokeh_figure=cp_plot)
# plotting the reserves before and after swap 
pool_plot = new_pool_figure(usdt_pool, uni_pool, steps=["L0", "L1", "L2", "L3"])
# display plots
show(column(cp_plot, pool_plot, sizing_mode="stretch_both"))

Constant Product AMM Order Book
---

From the paper [On Equivalence of Automated Market Maker and Limit Order Book Systems](https://professorjey.com/assets/papers/AMM_Order_Book_Equivalence_DRAFT_2020_10_16.pdf) the equation for the cumulative quantity at any mid price is given by:

$$ Q_{cum}(p)=\begin{cases}
\begin{aligned}
\ & x_{0}(\sqrt{\frac{P_{0}}{P}} - 1) & \text{if } p < p_{0} \newline
\ & 0 &\text{if } p = p_{0} \newline
\ & x_{0}(1 - \sqrt{\frac{P_{0}}{P}}) & \text{if } p < p_{0}
\end{aligned}
\end{cases} $$

In [7]:
from cpy_amm.swap import init_liquidity, MarketQuote
from cpy_amm.bokeh_swap import new_order_book_figure

# initial liquidity in USD
deposit_usd = 100000
# USDC/USD market price
usdt_usd = MarketQuote("USDT/USD", 1)
# ETH/USD market price
uni_usd = MarketQuote("UNI/USD", 6.32)
# initialize 2 pools given initial deposit and market quotes, all in USD
usdt_pool, uni_pool, k = init_liquidity(deposit_usd, usdt_usd, uni_usd)
# new plot with constant product curve
order_book_plot = new_order_book_figure(usdt_pool, uni_pool, x_min=20000, x_max=80000)
# display plots
show(column(order_book_plot, sizing_mode="stretch_both"))